# 6.14.1 Проект - решение


In [ ]:
import asyncio
import json
import re
import urllib.request
from typing import List, Dict, Any, Set

BANNED_URL = "https://raw.githubusercontent.com/nefelsay/asyncio/main/task_1_banned_word.txt"
MESSAGES_URL = "https://raw.githubusercontent.com/nefelsay/asyncio/main/task_1_message.json"

def load_banned_words() -> Set[str]:
    with urllib.request.urlopen(BANNED_URL, timeout=30) as resp:
        text = resp.read().decode("utf-8")
    return {line.strip().lower() for line in text.splitlines() if line.strip()}

def load_messages() -> List[Dict[str, Any]]:
    with urllib.request.urlopen(MESSAGES_URL, timeout=30) as resp:
        return json.loads(resp.read().decode("utf-8"))

def tokenize(text: str) -> List[str]:
    parts = text.lower().split()
    return [re.sub(r"^[^\wа-яё]+|[^\wа-яё]+$", "", p, flags=re.IGNORECASE) for p in parts]

async def check_message(message: Dict[str, Any], banned: Set[str]) -> None:
    mid = message["message_id"]
    text = message["message"]

    words = tokenize(text)
    found = next((w for w in words if w in banned), None)
    if found:
        task = asyncio.current_task()
        print(f"В сообщении {mid} стоп-слово: task.done(): {task.done()}")
        task.cancel()
        raise asyncio.CancelledError()

    print(f"{mid}: {text}")

async def main():
    banned = load_banned_words()
    messages = load_messages()
    tasks = [asyncio.create_task(check_message(m, banned)) for m in messages]
    await asyncio.gather(*tasks, return_exceptions=True)

asyncio.run(main())
